In [ ]:
# 在具体使用的时候，我们需要实例化一个对象
# 然后在以下语句的范围内进行模型的编译与定义即可
tf.distribute.MirroredStrategy()
with strategy.scope():
    # 模型的定义与编译
    ......

In [ ]:
'''
值得注意的是，这边的 Batch 大小：
    我们首先定义单个 GPU 上的批次大小；
    然后我们需要定义总体的批次大小 = 单个 GPU 上的批次大小 * GPU 数量；
    在训练过程之中，TensorFlow 会将批次大小自动分割分给不同的 GPU。
'''
import tensorflow as tf

# 获取fashion_mnist数据集
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# 预处理图片数据，使其归一化
x_train, x_test = x_train / 255.0, x_test / 255.0

# 定义镜像策略
strategy = tf.distribute.MirroredStrategy()

# 单个GPU之上的Batch数量
batch_per_gpu = 64
# 同时训练的Batch大小应该为：单个GPU的Batch * GPU个数
batch = batch_per_gpu * strategy.num_replicas_in_sync


train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch).shuffle(buffer_size=10000)
eval_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch)

# 在这里写网络模型
with strategy.scope():
  model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
  

model.fit(train_dataset, epochs=10)

eval_loss, eval_acc = model.evaluate(eval_dataset)
print(eval_loss, eval_acc)